In [1]:
import configparser
import csv
import glob
import logging
import re
from datetime import datetime
from db_connect import DatabaseConnection
from logging_config import set_logs

In [2]:
# для записи логов
logger = set_logs(datetime.now())

# чтение файла с данными для подключения
config = configparser.ConfigParser()
config.read("config.ini")
DATABASE_CREDS = config["Database"]
DATABASE=DATABASE_CREDS["DATABASE"]
USER=DATABASE_CREDS["USER"]
PASSWORD=DATABASE_CREDS["PASSWORD"]
HOST=DATABASE_CREDS["HOST"]
PORT=DATABASE_CREDS["PORT"]

# Поиск всех файлов .csv в папке data
files = glob.glob('data/*.csv')
# регулярное выражение для поиска нужных файлов
pattern = r'data\\(1[0-9]|20)-(1|2|3|4|5)\.csv$' 
with DatabaseConnection(DATABASE, USER, PASSWORD, HOST, PORT) as dbc:
        for file_path in files:
            if re.search(pattern, file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    csv_reader = csv.reader(file)
                    next(csv_reader)
                    data = list(csv_reader)
                insert_query = """
                INSERT INTO finance (doc_id, item, category,
                                    amount, price, discount)
                VALUES (%s, %s, %s, %s, %s, %s)
                """          
                # запись в БД    
                dbc.insert_data(insert_query, data, file_path)
# закрытие файла с логами
logger.close()